# Τεχνητή Νοημοσύνη: Θέμα 2

---


Τσιγγέλης Κωνσταντίνος
A.M.: 03117149

---

## Λύση

### Μέρος 1 ###  


#### Προετοιμασία του κόσμου του προβλήματος
---

#### Ερώτημα 1α  
Δημιουργία αρχείου ```db.pl```

In [1]:
import pandas as pd
from pyswip import Prolog
import json
path = './'

data = pd.read_csv(path + 'movies_metadata.csv')
data.fillna("UNK", inplace=True)

def clean_text(text):
  text = text.replace(u'\xa0', u'')
  text = text.replace(u"'", u'')
  return text

In [2]:
column_names = {'budget':'budget', 'genres': 'genre', 'plot_keywords':'keywords', 'language':'language', 'original_title':'originalTitle', 'popularity':'popularity', 'production_companies':'productionCompany', 'production_countries':'productionCountry', 'release_date':'releaseMonth', 'gross':'gross', 'duration':'duration', 'spoken_languages':'spokenLanguages', 'movie_title':'movieTitle', 'vote_average':'voteAverage', 'title_year':'titleYear', 'country':'country', 'director_name':'director', 'actor_1_name':'actor1', 'actor_2_name':'actor2', 'actor_3_name':'actor3'}
##
def doOneColumn(column, curfile):
    """
    This creates and inserts all facts of one collumn, of all lines 
    """
    literals = []
    for row in data.itertuples(index=True, name='Pandas'):
        movie_title = clean_text(getattr(row, 'movie_title'))
        
        currattr = clean_text(str(getattr(row,column)))
        for item in currattr.split("|"):
            literals.append(column_names[column]+"('"+ movie_title +"','"+ item +"')")
    literals.sort()
    for literal in literals:
        curfile.write(literal+"."+"\n")
##
def specialColumn(column, curfile):
    """
    This does same as above but also does the necessary changes due to the format of the 
    attributes production_companies and production_countries and spoken_languages
    """
    literals = []
    for row in data.itertuples(index=True, name='Pandas'):
        movie_title = clean_text(getattr(row, 'movie_title'))
    
        currattr = ((getattr(row,column)))
        # print(clean_text(currattr.replace("'", '"')))
        if '"' in currattr: continue
        currattr = json.loads(clean_text(currattr.replace("'", '"')))
        for company in currattr:
            company = clean_text(company["name"])
            literals.append(column_names[column]+"('"+ movie_title +"','"+ company +"')")
    literals.sort()
    for literal in literals:
        curfile.write(literal+"."+"\n")

def doRelease(curfile):
    literals = []
    for row in data.itertuples(index=True, name='Pandas'):
        movie_title = clean_text(getattr(row, 'movie_title'))

        currattr = (getattr(row,"release_date"))
        if currattr == "UNK": continue
        currattr = currattr.split("-")
        currattr = currattr[1]

        literals.append(column_names["release_date"]+"('"+ movie_title +"','"+ currattr +"')")
    literals.sort()
    for literal in literals:
        curfile.write(literal+"."+"\n")


##
def doAll():
    """
    This creates a file db.pl in the current directory
    """
    curfile = open("db.pl", "w",encoding="utf-8")
    for column in column_names:
        if column not in ["production_companies", "production_countries","spoken_languages","release_date"]:
            doOneColumn(column,curfile)
        elif column != 'release_date':
            specialColumn(column,curfile)
        else:
            doRelease(curfile)
    curfile.close()


Παρακάτω κελί δημιουργεί ένα αρχείο ```db.pl``` με τον κόσμο τους προβλήματος

In [3]:
doAll()

#### Ερώτημα 1β  
Δημιουργία αρχείου ```rules.pl```

In [4]:
f = open("rules.pl", 'w')
f.write(
'''
fiveGenresCommon(X,Y):-
  genre(X,N1), genre(X,N2), N1 \= N2,
  genre(Y,N1), genre(Y,N2), X \= Y,
  genre(X,N3), N1 \= N3, N2 \= N3,
  genre(Y,N3),
  genre(X,N4), N1 \= N4, N2 \= N4, N3 \= N4,
  genre(Y,N4),
  genre(X,N5), N1 \= N5, N2 \= N5,
  N3 \= N5, N4 \= N5,
  genre(Y,N5).

fourGenresCommon(X,Y):-
  genre(X,N1), genre(X,N2), N1 \= N2,
  genre(Y,N1), genre(Y,N2), X \= Y,
  genre(X,N3), N1 \= N3, N2 \= N3,
  genre(Y,N3), genre(X,N4),
  N1 \= N4, N2 \= N4, N3 \= N4,
  genre(Y,N4).

threeGenresCommon(X,Y):-
  genre(X,N1), genre(X,N2), genre(X,N3),
  N1 \= N2, N2 \= N3, N1 \= N3,
  genre(Y,N1), genre(Y,N2), genre(Y,N3),
  X \= Y.

twoGenresCommon(X,Y):-
  genre(X,N1), genre(X,N2),
  N1 \= N2,
  genre(Y,N1), genre(Y,N2),
  X \= Y.

oneGenreCommon(X,Y):-
  genre(X,N), genre(Y,N), X \= Y.

commonDirector(X,Y):-
  director(X,N), director(Y,N), X \= Y.

samePlot(X,Y):-
  keywords(X,N1), keywords(X,N2), N1 \= N2,
  keywords(Y,N1), keywords(Y,N2), X \= Y,
  keywords(X,N3), N1 \= N3, N2 \= N3,
  keywords(Y,N3),
  keywords(X,N4), N1 \= N4, N2 \= N4, N3 \= N4,
  keywords(Y,N4),
  keywords(X,N5), N1 \= N5,N2 \= N5,
  N3 \= N5, N4 \= N5,
  keywords(Y,N5).

fourKeywords(X,Y):-
  keywords(X,N1), keywords(X,N2), N1 \= N2,
  keywords(Y,N1), keywords(Y,N2), X \= Y,
  keywords(X,N3), N1 \= N3, N2 \= N3,
  keywords(Y,N3),
  keywords(X,N4), N1 \= N4, N2 \= N4, N3 \= N4,
  keywords(Y,N4).

threeKeywords(X,Y):-
  keywords(X,N1), keywords(X,N2),
  N1 \= N2,
  keywords(Y,N1), keywords(Y,N1),
  X \= Y, keywords(X,N3),
  N1 \= N3, N2 \= N3,
  keywords(Y,N3).

similarPlot(X,Y):-
  keywords(X,N1), keywords(X,N2),
  N1 \= N2,
  keywords(Y,N1), keywords(Y,N1),
  X \= Y.

oneKeyword(X,Y):-
  keywords(X,N), keywords(Y,N), X \= Y.

actor(X,N):-actor1(X,N).
actor(X,N):-actor2(X,N).
actor(X,N):-actor3(X,N).

sameActors(X,Y):-
  actor(X,N1), actor(X,N2), actor(X,N3),
  N1 \= N2, N1 \= N3, N2 \= N3,
  actor(Y,N1), actor(Y,N2), actor(Y,N3),
  X \= Y.

similarActors(X,Y):-
  actor(X,N1), actor(X,N2), N1 \= N2,
  actor(Y,N1), actor(Y,N2), X \= Y.

oneCommonActor(X,Y):-
  actor(X,N), actor(Y,N), X \= Y.

sameLanguage(X,Y):-
  language(X,N), language(Y,N), X \= Y.

blackAndWhite(X):-keywords(X,'black and white').

commonCompany(X,Y):-
  productionCompany(X,N),
  productionCompany(Y,N),
  X \= Y.

commonCountry(X,Y):-
  productionCountry(X,N),
  productionCountry(Y,N),
  x \= Y.

winterRelease(X):-releaseMonth(X,'12').
winterRelease(X):-releaseMonth(X,'01').
winterRelease(X):-releaseMonth(X,'02').

springRelease(X):-releaseMonth(X,'03').
springRelease(X):-releaseMonth(X,'04').
springRelease(X):-releaseMonth(X,'05').

summerRelease(X):-releaseMonth(X,'06').
summerRelease(X):-releaseMonth(X,'07').
summerRelease(X):-releaseMonth(X,'08').

autumnRelease(X):-releaseMonth(X,'09').
autumnRelease(X):-releaseMonth(X,'10').
autumnRelease(X):-releaseMonth(X,'11').

sameSeason(X,Y):- autumnRelease(X),autumnRelease(Y),X\=Y.
sameSeason(X,Y):- winterRelease(X),winterRelease(Y),X\=Y.
sameSeason(X,Y):- summerRelease(X),summerRelease(Y),X\=Y.
sameSeason(X,Y):- springRelease(X),springRelease(Y),X\=Y.


commonLanguages(X,Y):-
  spokenLanguages(X,N1),
  N1 \= 'English',
  spokenLanguages(Y,N1),
  X \= Y.

highlyVoted(X):-
  voteAverage(X,A),
  atom_number(A,N),
  N > 7.

closeNumbers(Test):-
  Test < 7000000, Test > -7000000.
closeBudget(A1,A2):-
  atom_number(A1,N1),
  atom_number(A2,N2),
  Test is N1 - N2,
  closeNumbers(Test).
similarBudget(X,Y):-
  budget(X,A1), budget(Y,A2), X \= Y,
  closeBudget(A1,A2).

releasedThisDecade(X):-
  titleYear(X,A),
  atom_number(A,N),
  N < 2021,
  N > 2014.

releasedLastDecade(X):-
  titleYear(X,A),
  atom_number(A,N),
  N < 2015,
  N > 2009.

releasedYeOldenTimes(X):-
  titleYear(X,A),
  atom_number(A,N),
  N < 2010,
  N > 2004.

testRelease1(X):-
  titleYear(X,A),
  atom_number(A,N),
  N < 2005,
  N > 1999.

testRelease2(X):-
  titleYear(X,A),
  atom_number(A,N),
  N < 2000,
  N > 1994.
  
testRelease3(X):-
  titleYear(X,A),
  atom_number(A,N),
  N < 1995.

samePeriod(X,Y):- releasedThisDecade(X), releasedThisDecade(Y), X\=Y.
samePeriod(X,Y):- releasedLastDecade(X), releasedLastDecade(Y), X\=Y.
samePeriod(X,Y):- releasedYeOldenTimes(X), releasedYeOldenTimes(Y), X\=Y.
samePeriod(X,Y):- testRelease1(X), testRelease1(Y), X\=Y.
samePeriod(X,Y):- testRelease2(X), testRelease2(Y), X\=Y.
samePeriod(X,Y):- testRelease3(X), testRelease3(Y), X\=Y.

'''
)
f.close()

#### Επίδειξη λειτουργίας Μέρους 1

In [5]:
from pyswip import Prolog
prolog = Prolog()
prolog.consult("db.pl")       # change this 
prolog.consult("rules.pl")    # and this, accordingly 

In [6]:
sampleMovies = ["Avatar", "The Dark Knight Rises", "Cutthroat Island", "Men in Black", 
                "The Phantom of the Opera"]
sampleQueries = ["threeGenresCommon", "oneGenreCommon", "commonDirector", "samePlot",
                "sameActors", "commonCompany"]
# test = []
# temp = (prolog.query("highlyVoted(X)"))
for movie in sampleMovies:
  print("\033[31m -----------------------------------------------------------------------------------------------------------------------------\33[0m")
  print("Movie: "+movie)
  for query in sampleQueries:
    q = prolog.query(query+"('"+movie+"',X)")
    print("   Query: "+query)
    temp = [x["X"] for x in q]
    s = set()
    for i in temp: 
      if i not in s: s.add(i)
    print("   Some of the \033[31m"+str(len(s))+"\33[0m are: "+str(list(s)[:10]))

 -----------------------------------------------------------------------------------------------------------------------------
Movie: Avatar
   Query: threeGenresCommon
   Some of the 237 are: ['John Carter', 'Captain America: Civil War', 'Congo', 'After Earth', 'The Lord of the Rings: The Two Towers', 'Aliens vs Predator: Requiem', 'Mighty Joe Young', 'Return of the Jedi', 'Moonraker', 'Enders Game']
   Query: oneGenreCommon
   Some of the 1827 are: ['Its a Mad, Mad, Mad, Mad World', 'Ladder 49', 'Lone Wolf McQuade', 'Machine Gun McCain', 'Aliens vs Predator: Requiem', 'Sabotage', 'Edward Scissorhands', 'The Last Castle', 'Space Battleship Yamato', 'The Prince']
   Query: commonDirector
   Some of the 6 are: ['Terminator 2: Judgment Day', 'Titanic', 'Aliens', 'The Terminator', 'The Abyss', 'True Lies']
   Query: samePlot
   Some of the 0 are: []
   Query: sameActors
   Some of the 0 are: []
   Query: commonCompany
   Some of the 253 are: ['The Martian', 'Hope Floats', 'The Legend of B

In [7]:
for query in ["winterRelease", "blackAndWhite", "highlyVoted"]:
  q = prolog.query(query+"(X)")
  print("\033[31m -----------------------------------------------------------------------------------------------------------------------------\33[0m")
  print("Query: "+query)
  temp = [x["X"] for x in q]
  s = set()
  for i in temp: 
    if i not in s: s.add(i)
  print("   Some of the \033[31m"+str(len(s))+"\33[0m are: "+str(list(s)[:10]))

 -----------------------------------------------------------------------------------------------------------------------------
Query: winterRelease
   Some of the 1164 are: ['Butterfly', 'L.I.E.', 'Spice World', '3 Backyards', 'Maniac', 'Brooklyns Finest', 'Coriolanus', 'Harts War', 'Aliens vs Predator: Requiem', 'Oceans Twelve']
 -----------------------------------------------------------------------------------------------------------------------------
Query: blackAndWhite
   Some of the 6 are: ['Frances Ha', 'Renaissance', 'Frankenweenie', 'Creative Control', 'The Giver', 'The Phantom of the Opera']
 -----------------------------------------------------------------------------------------------------------------------------
Query: highlyVoted
   Some of the 809 are: ['Its All Gone Pete Tong', 'The Conversation', 'Eraserhead', 'The Messenger', 'Harry Potter and the Philosophers Stone', 'Machine Gun McCain', 'Trading Places', 'Ponyo', 'Edward Scissorhands', 'Little Miss Sunshine']


### Μέρος 2 ###  


#### Σύστημα προτάσεων
---

#### Δημιουργία αρχείου ```recommendations.pl```

In [8]:
f = open("recommendations.pl", "w")
f.write(
'''
findSim1(X,Y):- fourKeywords(X,Y). 
findSim1(X,Y):- similarActors(X,Y). 
%findSim1(X,Y):- highlyVoted(X), highlyVoted(Y), X\=Y. 
findSim1(X,Y):- blackAndWhite(X), blackAndWhite(Y), X\=Y. 
findSim1(X,Y):- commonDirector(X,Y).  
findSim1(X,Y):- commonCompany(X,Y).
findSim1(X,Y):- commonLanguages(X,Y).
findSim1(X,Y):- fourGenresCommon(X,Y).
findSim1(X,Y):- samePeriod(X,Y).

findSim2(X,Y):- fourKeywords(X,Y).
findSim2(X,Y):- similarActors(X,Y).
findSim2(X,Y):- highlyVoted(X), highlyVoted(Y), X\=Y.
findSim2(X,Y):- blackAndWhite(X), blackAndWhite(Y), X\=Y.
findSim2(X,Y):- commonDirector(X,Y). 
findSim2(X,Y):- commonCompany(X,Y).
findSim2(X,Y):- commonLanguages(X,Y).
findSim2(X,Y):- fourGenresCommon(X,Y).


findSim3(X,Y):- fourKeywords(X,Y).
findSim3(X,Y):- similarActors(X,Y).
findSim3(X,Y):- highlyVoted(X), highlyVoted(Y), X\=Y, similarBudget(X,Y).
findSim3(X,Y):- blackAndWhite(X), blackAndWhite(Y), X\=Y.
findSim3(X,Y):- commonDirector(X,Y). 
findSim3(X,Y):- commonCompany(X,Y).
findSim3(X,Y):- commonLanguages(X,Y).
findSim3(X,Y):- fourGenresCommon(X,Y).

findSim4(X,Y):- fourKeywords(X,Y).
findSim4(X,Y):- sameActors(X,Y).
findSim4(X,Y):- blackAndWhite(X), blackAndWhite(Y), X\=Y.
findSim4(X,Y):- commonDirector(X,Y).
findSim4(X,Y):- commonCompany(X,Y).
findSim4(X,Y):- fiveGenresCommon(X,Y).

findSim5(X,Y):- samePlot(X,Y).
findSim5(X,Y):- sameActors(X,Y).
findSim5(X,Y):- blackAndWhite(X), blackAndWhite(Y), X\=Y.
findSim5(X,Y):- commonDirector(X,Y).
findSim5(X,Y):- fiveGenresCommon(X,Y).
'''
)
f.close()

#### Προετοιμασία κόσμου και κανόνων και εισαγωγή φίλτρων στον κόσμο

In [9]:
from pyswip import Prolog

prolog = Prolog()
prolog.consult("db.pl")
prolog.consult("rules.pl")
prolog.consult("recommendations.pl")

#### Υλοποίηση της ```simple_recommender``` 

In [10]:
def simple_recommender(movie, precision=1):
  s = set()
  q = prolog.query("findSim"+ str(precision) +"('"+ movie +"',Movie)")
  for item in q:
    m = item["Movie"]
    if m not in s: s.add(m)
  q.close()
  answers = s
  return answers

#### Επίδειξη λειτουργίας Μέρους 

In [11]:
sampleMovies = ['Avatar','The Dark Knight Rises','Cutthroat Island',
                'Men in Black','The Phantom of the Opera',
                'Lara Croft: Tomb Raider', 'I Think I Love My Wife', 'The Best Offer',  'Astro Boy',
                'Two Evil Eyes']
for movie in sampleMovies:
  print("\033[95m -----------------------------------------------------------------------------------------------------------------------------\33[0m")
  print("Movie: "+ movie)
  for i in range(1,6):
    print("   Precision:  "+ str(i))
    temp = list(simple_recommender(movie,i))
    print("         \033[91mAmount\33[0m:   "+ str(len(temp)))
    print("         \033[91mSome are\33[0m: "+ str(temp[:10]))

 -----------------------------------------------------------------------------------------------------------------------------
Movie: Avatar
   Precision:  1
         Amount:   1568
         Some are: ['Lone Wolf McQuade', 'Aliens vs Predator: Requiem', 'Edward Scissorhands', 'Diary of a Mad Black Woman', 'Little Miss Sunshine', 'Paa', 'Victor Frankenstein', 'Fantastic Four', 'Glee: The Concert Movie', 'Like Mike']
   Precision:  2
         Amount:   1291
         Some are: ['Lone Wolf McQuade', 'Machine Gun McCain', 'Aliens vs Predator: Requiem', 'Trading Places', 'Edward Scissorhands', 'Little Miss Sunshine', 'Victor Frankenstein', 'Downfall', 'Fantastic Four', 'Glee: The Concert Movie']
   Precision:  3
         Amount:   602
         Some are: ['Casa De Mi Padre', 'Hope Floats', 'Dont Say a Word', '30 Minutes or Less', 'Live Free or Die Hard', 'Lone Wolf McQuade', 'The Messenger', 'Aliens vs Predator: Requiem', 'Beneath the Planet of the Apes', 'Edward Scissorhands']
   Precision: 

         Amount:   1389
         Some are: ['Cheap Thrills', '50/50', 'Roadside', 'Oceans Twelve', 'Sabotage', 'Space Battleship Yamato', 'The Prince', 'Atlas Shrugged Part III: Who is John Galt?', 'Wicked Blood', 'Glee: The Concert Movie']
   Precision:  2
         Amount:   1047
         Some are: ['Red Planet', 'Its All Gone Pete Tong', 'The Conversation', 'Eraserhead', 'The Messenger', 'Harry Potter and the Philosophers Stone', 'Machine Gun McCain', 'Whiteout', 'Trading Places', 'Ponyo']
   Precision:  3
         Amount:   487
         Some are: ['Red Planet', 'Harry Potter and the Philosophers Stone', 'Whiteout', 'Contagion', 'Oceans Twelve', 'Edward Scissorhands', 'Little Miss Sunshine', 'Home Alone', 'Downfall', 'Frida']
   Precision:  4
         Amount:   315
         Some are: ['Red Planet', 'Harry Potter and the Philosophers Stone', 'Whiteout', 'Contagion', 'Oceans Twelve', 'Malcolm X', 'Three Kings', 'Conspiracy Theory', 'Scooby-Doo 2: Monsters Unleashed', 'The Fountain']
  

### Μέρος 3  

#### Σύστημα προτάσεων με βάση τις προτιμήσεις-αξιολογήσεις του χρήστη
#### Εκπαίδευση και πρόβλεψη
---

In [12]:
import pandas as pd
from pyswip import Prolog
import json
prolog = Prolog()
prolog.consult("db.pl")
prolog.consult("rules.pl")
prolog.consult("recommendations.pl")

In [13]:
def clean_text(text):
  text = text.replace(u'\xa0', u'')
  text = text.replace(u"'", u'')
  return text
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

#placing here for easier access
def simple_recommender(movie, precision=1):
  #print("\nCurrently doing movie '"+movie+"' with precision '"+str(precision)+"'")
  s = set()
  q = prolog.query("findall(N,findSim"+ str(precision) +"('"+ movie +"',N), Movie)")
  for item in next(q)["Movie"]:
    if str(item) not in s: s.add(str(item))
  q.close()
  answers = s
#   print("Exiting with list length: " + str(len(list(answers))), end=' |||| ')
  return answers

rating_weights = {0: -1, 1: -0.5, 2:0, 3:0, 4:0.5, 5:1}
#score_weights = {(i+1):(i+1) for i in range(5)} # ανάλογα με τα επίπεδα ομοιότητας που έχουν οριστεί στην simple_recommender
score_weights = {1:0.0001, 2: 0.01, 3: 3, 4: 6, 5: 8}

def train_recommender(ratings, rating_weights, score_weights, start=0, end = -1, movie_score = {}):
    """
    Στην συνάρτηση αυτή μπορούμε να ορίζουμε ποιο υποσύνολο των αξιολογήσεων θα χρησιμοποιήσουμε για το train,
    ενώ παράλληλα μπορούμε να αρχικοποιήσουμε και το movie_score το οποίο αποθηκεύονται τα score κάθε ταινίας.
    Έτσι αν έχουμε κάνει train τον recommender μας για τις πρώτες 10 ταινίες, μπορούμε να συνεχίσουμε για τις 
    υπόλοιπες χωρίς κάθε φορά να ξανά-διαμορφώνουμε το score για αυτές. Αυτό θα σας βοηθήσει στην εκτέλεση και χρονικά όταν ελέγχετε την 
    απόδοση σε υποσύνολα των ratings π.χ. πρώτες 10 ταινίες έπειτα τις επόμενες 20 κ.ο.κ.
    """
    if end == -1:
        end = len(ratings)

    ratings = ratings.iloc[range(start, end)]
    for row in tqdm(ratings.itertuples(index=True, name='Pandas'), ncols='50%'):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        
        similar_movies = {i:simple_recommender(movie,i) for i in range(5,0,-1)}
        doneMovies = set()
        #print("\033[92m Right after simple_recommenders \033[0m")

        for precision in range(5,0,-1):
          for similar_movie in similar_movies[precision]:
              if similar_movie in doneMovies: continue
              else: doneMovies.add(similar_movie)
              if similar_movie not in movie_score:
                  movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[precision]
              else:
                  movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[precision] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    return movie_score


#αυτό είναι ένα παράδειγμα για το πως θα μπορούσε να υλοποιήθει η predict
#έχουμε ορίσει ότι μια ταινία θα έπρεπε να είναι προτεινόμενη αν είχε σκορ > 0
def predict_example(ratings, movie_score, heur=0):
    real, pred = [], []
    for i, row in enumerate(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        if movie in movie_score: #αν έχουμε σχηματίσει βαθμολογία για την ταινία αυτή
            pred.append(int(movie_score[movie] > -0.2)) #heuristic για το αν μια ταινία είναι προτεινόμενη
            real.append(int(rating > 3))# έτσι ορίζουμε ότι μια ταινία θα έπρεπε να είναι προτεινόμενη
            #η συνθήκη αυτή δεν μπορεί να αλλάξει
        else: #δεν μπορούμε να προτείνουμε κάτι για το όποιο δεν έχουμε σχηματίσει εικόνα
            pred.append(0)
            real.append(int(rating > 3))

    return real, pred


def get_metrics(real, pred):
    metrics = {}
    metrics["precision"] = precision_score(real, pred)
    metrics["recall"] = recall_score(real, pred)
    metrics["f1"] = f1_score(real, pred)
    return metrics

def full_tester():
    '''
    Αυτή χρησιμοποιήθηκε για γρηγορότερο έλεγχο διαφόρων βαρών και ευριστικών που
    δοκιμάσαμε
    '''
    for i in range(0,91,60):
        print("\nDoing from "+ str(i) +" to "+ str(i+10))
        movie_score = {}
        movie_score = train_recommender(train_ratings, rating_weights, score_weights, i, i+10)
        real, pred = predict_example(test_ratings, movie_score)
        print(get_metrics(real, pred))
        print("\n------------------------------------------------------")
    for i in range(0,91,50):
        print("\nDoing from "+ str(i) +" to "+ str(i+50))
        movie_score = {}
        movie_score = train_recommender(train_ratings, rating_weights, score_weights, i, i+50)
        real, pred = predict_example(test_ratings, movie_score)
        print(get_metrics(real, pred))
        print("\n------------------------------------------------------")
        
    print("\nDoing all (0 to 100)")
    movie_score = {}
    movie_score = train_recommender(train_ratings, rating_weights, score_weights, start=0)
    real, pred = predict_example(test_ratings, movie_score)
    print(get_metrics(real,pred))

def heuristic_tester():
    '''
    Αυτή χρησιμοποιήθηκε συγκεκριμένα για την εύκολη δοκιμή διάφορων ευριστικών
    '''
    for i in range (0, 0.2, 0.02):
        print("Heuristic: "+ str(i))
        movie_score = {}
        movie_score = train_recommender(train_ratings, rating_weights, score_weights, start=10, end=50)
        real, pred = predict_example(test_ratings, movie_score)
        print(get_metrics(real, pred))
        print("\n------------------------------------------------------")
        movie_score = {}
        movie_score = train_recommender(train_ratings, rating_weights, score_weights, start=30, end=70)
        real, pred = predict_example(test_ratings, movie_score)
        print(get_metrics(real, pred))
        print("\n------------------------------------------------------")


In [14]:
train_ratings = pd.read_csv("train_ratings.csv")
test_ratings = pd.read_csv("test_ratings.csv")

In [15]:
full_tester()


Doing from 0 to 10



{'precision': 0.5190839694656488, 'recall': 0.9444444444444444, 'f1': 0.6699507389162561}

------------------------------------------------------

Doing from 60 to 70



{'precision': 0.5081967213114754, 'recall': 0.8611111111111112, 'f1': 0.6391752577319588}

------------------------------------------------------

Doing from 0 to 50



{'precision': 0.518796992481203, 'recall': 0.9583333333333334, 'f1': 0.6731707317073171}

------------------------------------------------------

Doing from 50 to 100



{'precision': 0.5234375, 'recall': 0.9305555555555556, 'f1': 0.6699999999999999}

------------------------------------------------------

Doing all (0 to 100)



{'precision': 0.5234375, 'recall': 0.9305555555555556, 'f1': 0.6699999999999999}


## Συμπεράσματα
---

Δοκιμάζοντας μεγάλο πλήθος συνδιασμών και στα score_weights και στο όριο πάνω απ'το οποίο προτείνουμε μια ταινία, αλλά κυρίως στους κανόνες findSim δεν καταφέραμε να δούμε μεγάλη διαφορά στις μετρικές μας. Το παρόν αποτέλεσμα δείχνει ότι το σύστημά μας προτείνει λογικά περισσότερες ταινίες απ'όσες έπρεπε τις περισσότερες φορές.  

Προσπαθήσαμε να βελτιώσουμε την ακρίβειά του κάνοντας πιό αυστηρές τις findSim αλλά δεν υπήρξε πάλι μεγάλη διαφορά. Αυτό βέβαια δε σημαίνει ότι θα ισχύει και το ίδιο άλλους πιθανούς χρήστες

Το επόμενο βήμα αν θέλαμε (και προλαβαίναμε) να βελτιώσουμε την απόδοση του συστήματος, θα ήταν να φτιάξουμε ένα υποσύστημα που να δοκιμάζει αυτόματα αρκετούς συνδιασμούς ευριστικών ορίων, βαρών που αποδίδονται στους βαθμούς ομοιότητας και κυριότερα, διαφόρων αυτόματα δημιουργημένων findSim κανόνων και να παρατηρεί αποτελέσματα.